# Data Engineering Project

## What problem will we solve? ##
We will look at the most populous U.S. cities and identify which ones are the most expensive and the most affordable to live in. This will help us decide which city we'd like to move to next.

## What datasets will we use? ##

We will scrape three datasets: <br/>

1) Wikipedia [List of United States cities by population](https://en.wikipedia.org/wiki/List_of_United_States_cities_by_population) data, which lists the most populous U.S. cities

2) [Zillow Home Value Index](https://www.zillow.com/research/zhvi-methodology-6032/) data, an estimate of median home values by city. 

3) Wikipedia [Household income in the United States](https://en.wikipedia.org/wiki/Household_income_in_the_United_States) data, which lists 2017 median household income by state 


    
    

## How will we use these datasets to solve the problem? ##
We will append (2) median home value data and (3) household income data to the (1) list of most populous U.S. cities, and then calculate a "Cost Score", which shows for each city how many years of income is required to purchase a median value home. This tells us, relative to other cities, how costly it is to live in a particular city. We will then create a "Cost Rank" based on this score. </br>

"Cost Score" equation is:


$$\mathtt Cost Score= \frac{Median Home Value}{Median Income}$$

If "Cost Score" = 1,000,000 / 100,000 = 10, this means it takes 10 years of median income to purchase a median value home.

## What steps will we take to do this? ##

We will: <br/>

I. Scrape the Data <br/>

II. Join the Data <br/>

III. Analyze the Data

## Step I. Scrape the Data

### First, let's download (1) Wikipedia 'List of United States cities by population' data.

In [130]:
# import libraries
import requests
import urllib.request
import time
from bs4 import BeautifulSoup # great for parsing html
import pandas as pd
import wikipedia as wp # great for fetching wikipedia html data

In [662]:
# set variable for wikipedia page html
html = wp.page('List of United States cities by population').html().encode("UTF-8")

# set variable for our main data table. the data table we want to fetch is the fourth element in the wikipedia page html
df = pd.read_html(html)[4] 

In [663]:
# display dataframe
df.head()

,2018rank,City,State[c],2018estimate,2010Census,Change,2016 land area,2016 land area.1,2016 population density,2016 population density.1,Location
0,1,New York City[d],New York,8398748,8175133,+2.74%,301.5 sq mi,780.9 km2,"28,317/sq mi","10,933/km2",40Â°39â²49â³N 73Â°56â²19â³W﻿ / ﻿40.6635Â°N...
1,2,Los Angeles,California,3990456,3792621,+5.22%,468.7 sq mi,"1,213.9 km2","8,484/sq mi","3,276/km2",34Â°01â²10â³N 118Â°24â²39â³W﻿ / ﻿34.0194Â°...
2,3,Chicago,Illinois,2705994,2695598,+0.39%,227.3 sq mi,588.7 km2,"11,900/sq mi","4,600/km2",41Â°50â²15â³N 87Â°40â²54â³W﻿ / ﻿41.8376Â°N...
3,4,Houston[3],Texas,2325502,2100263,+10.72%,637.5 sq mi,"1,651.1 km2","3,613/sq mi","1,395/km2",29Â°47â²12â³N 95Â°23â²27â³W﻿ / ﻿29.7866Â°N...
4,5,Phoenix,Arizona,1660272,1445632,+14.85%,517.6 sq mi,"1,340.6 km2","3,120/sq mi","1,200/km2",33Â°34â²20â³N 112Â°05â²24â³W﻿ / ﻿33.5722Â°...


The following code checks for NaN or blank values. As we can see, there are none, and that is great.

In [245]:
# let's check for NaN or blank values
for i in df:
    print(i, ': ', df[i].isnull().values.any())

2018rank :  False
City :  False
State[c] :  False
2018estimate :  False
2010Census :  False
Change :  False
2016 land area :  False
2016 land area.1 :  False
2016 population density :  False
2016 population density.1 :  False
Location :  False


Examining the data types of our fields or columns, it appears that '2016 land area' and '2016 population density' are numeric values being stored as object, or string values. Let's update that. 

Also, some of the 'City' field values contain an annotation--let's remove these annotations.

In [664]:
df.dtypes # let's look at data types

2018rank                      int64
City                         object
State[c]                     object
2018estimate                  int64
2010Census                    int64
Change                       object
2016 land area               object
2016 land area.1             object
2016 population density      object
2016 population density.1    object
Location                     object
dtype: object

In [665]:
# convert to float
df['2016 land area'] = df['2016 land area'].apply(lambda x: x.split()[0].replace(',', '')).astype('float')
df['2016 population density'] = df['2016 population density'].apply(lambda x: x.split("/")[0].replace(',', '')).astype('float')

# get rid of the 'City' annotations
df['City'] = df['City'].apply(lambda x: x.split("[")[0]) #

In [666]:
df.head()

,2018rank,City,State[c],2018estimate,2010Census,Change,2016 land area,2016 land area.1,2016 population density,2016 population density.1,Location
0,1,New York City,New York,8398748,8175133,+2.74%,301.5,780.9 km2,28317.0,"10,933/km2",40Â°39â²49â³N 73Â°56â²19â³W﻿ / ﻿40.6635Â°N...
1,2,Los Angeles,California,3990456,3792621,+5.22%,468.7,"1,213.9 km2",8484.0,"3,276/km2",34Â°01â²10â³N 118Â°24â²39â³W﻿ / ﻿34.0194Â°...
2,3,Chicago,Illinois,2705994,2695598,+0.39%,227.3,588.7 km2,11900.0,"4,600/km2",41Â°50â²15â³N 87Â°40â²54â³W﻿ / ﻿41.8376Â°N...
3,4,Houston,Texas,2325502,2100263,+10.72%,637.5,"1,651.1 km2",3613.0,"1,395/km2",29Â°47â²12â³N 95Â°23â²27â³W﻿ / ﻿29.7866Â°N...
4,5,Phoenix,Arizona,1660272,1445632,+14.85%,517.6,"1,340.6 km2",3120.0,"1,200/km2",33Â°34â²20â³N 112Â°05â²24â³W﻿ / ﻿33.5722Â°...


Let's update the headers to reflect our conversions and let's also remove the annotation from the 'State[c]' header.

In [667]:
df.rename(columns = {"State[c]": "State", 
                     "2016 land area": "2016 land area (sq mi)", 
                     "2016 population density": "2016 population density (sq mi)"}, inplace = True)

We don't need 'Change', '2016 land area.1', '2016 population density.1', and 'Location' for our analysis, so let's drop these fields. 

In [668]:
df = df.drop(columns = ['Change', '2016 land area.1', '2016 population density.1', 'Location'])

Let's save our dataframe to csv.

In [669]:
df.to_csv('topCities.csv',header=1,index=False) # save to csv

In [670]:
df_topCities = pd.read_csv('topCities.csv') # read csv for further manipulation

### Next, let's download (2) 'Zillow Home Value Index' data.

In [672]:
# download zillow home index data
urllib.request.urlretrieve('http://files.zillowstatic.com/research/public/City/City_Zhvi_Summary_AllHomes.csv', './City_Zhvi_Summary_AllHomes.csv')

('./City_Zhvi_Summary_AllHomes.csv', <http.client.HTTPMessage at 0x11b904f98>)

In [674]:
# load zillow home index data. RegionName = city
df = pd.read_csv('City_Zhvi_Summary_AllHomes.csv', encoding='latin-1')

In [675]:
# display dataframe
df.head()

,Date,RegionID,RegionName,State,Metro,County,SizeRank,Zhvi,MoM,QoQ,YoY,5Year,10Year,PeakMonth,PeakQuarter,PeakZHVI,PctFallFromPeak,LastTimeAtCurrZHVI
0,2019-04-30,6181,New York,NY,New York-Newark-Jersey City,Queens County,0,677000,-0.002799,-0.003386,0.024051,0.062360,0.039604,2018-12,2018-Q4,680400,-0.004997,2018-10
1,2019-04-30,12447,Los Angeles,CA,Los Angeles-Long Beach-Anaheim,Los Angeles County,1,686100,-0.003775,-0.007809,0.021590,0.072236,0.049628,2019-01,2019-Q1,691500,-0.007809,2018-10
2,2019-04-30,39051,Houston,TX,Houston-The Woodlands-Sugar Land,Harris County,2,186000,-0.006410,0.001076,0.053824,0.081350,0.040710,2019-03,2019-Q1,187200,-0.006410,2019-03
3,2019-04-30,17426,Chicago,IL,Chicago-Naperville-Elgin,Cook County,3,229400,-0.002609,0.003939,0.005699,0.045435,0.004467,2007-02,2007-Q1,263900,-0.130731,2004-11
4,2019-04-30,6915,San Antonio,TX,San Antonio-New Braunfels,Bexar County,4,176100,-0.002831,0.001137,0.058930,0.075288,0.037982,2019-03,2019-Q1,176600,-0.002831,2019-02


The dataset contains NaN or blank values, but none for the fields we are focused on, 'RegionName' or 'City', and 'Zhvi', so we are good to go.

In [676]:
# let's check for NaN or blank values
for i in df:
    print(i, ': ', df[i].isnull().values.any())

Date :  False
RegionID :  False
RegionName :  False
State :  False
Metro :  True
County :  False
SizeRank :  False
Zhvi :  False
MoM :  False
QoQ :  False
YoY :  False
5Year :  True
10Year :  True
PeakMonth :  False
PeakQuarter :  False
PeakZHVI :  False
PctFallFromPeak :  False
LastTimeAtCurrZHVI :  True


In [262]:
df.dtypes # let's look at data types

Date                   object
RegionID                int64
RegionName             object
State                  object
Metro                  object
County                 object
SizeRank                int64
Zhvi                    int64
MoM                   float64
QoQ                   float64
YoY                   float64
5Year                 float64
10Year                float64
PeakMonth              object
PeakQuarter            object
PeakZHVI                int64
PctFallFromPeak       float64
LastTimeAtCurrZHVI     object
dtype: object

In [677]:
# let's keep only the columns we're interested in
df = df[['RegionName', 'State', 'Zhvi']]

In [678]:
# let's save to csv
df.to_csv('zhviCities.csv',header=1,index=False)

In [679]:
# df_zhviCities = pd.read_csv()
df_zhviCities = pd.read_csv('zhviCities.csv')

The 'State' field is abbreviated--we will need to convert this field to be spelled out. To do so, we will do a left join to connect a spelled out 'State' field to the abbreviated 'State' field.

In [688]:
# let's create a dataframe to convert state abbreviations to the full spelling
states = {
        'AK': 'Alaska',
        'AL': 'Alabama',
        'AR': 'Arkansas',
        'AS': 'American Samoa',
        'AZ': 'Arizona',
        'CA': 'California',
        'CO': 'Colorado',
        'CT': 'Connecticut',
        'DC': 'District of Columbia',
        'DE': 'Delaware',
        'FL': 'Florida',
        'GA': 'Georgia',
        'GU': 'Guam',
        'HI': 'Hawaii',
        'IA': 'Iowa',
        'ID': 'Idaho',
        'IL': 'Illinois',
        'IN': 'Indiana',
        'KS': 'Kansas',
        'KY': 'Kentucky',
        'LA': 'Louisiana',
        'MA': 'Massachusetts',
        'MD': 'Maryland',
        'ME': 'Maine',
        'MI': 'Michigan',
        'MN': 'Minnesota',
        'MO': 'Missouri',
        'MP': 'Northern Mariana Islands',
        'MS': 'Mississippi',
        'MT': 'Montana',
        'NA': 'National',
        'NC': 'North Carolina',
        'ND': 'North Dakota',
        'NE': 'Nebraska',
        'NH': 'New Hampshire',
        'NJ': 'New Jersey',
        'NM': 'New Mexico',
        'NV': 'Nevada',
        'NY': 'New York',
        'OH': 'Ohio',
        'OK': 'Oklahoma',
        'OR': 'Oregon',
        'PA': 'Pennsylvania',
        'PR': 'Puerto Rico',
        'RI': 'Rhode Island',
        'SC': 'South Carolina',
        'SD': 'South Dakota',
        'TN': 'Tennessee',
        'TX': 'Texas',
        'UT': 'Utah',
        'VA': 'Virginia',
        'VI': 'Virgin Islands',
        'VT': 'Vermont',
        'WA': 'Washington',
        'WI': 'Wisconsin',
        'WV': 'West Virginia',
        'WY': 'Wyoming'
}

In [689]:
df_states = pd.DataFrame(list(states.items()), columns=['Abbr State', 'State'])

In [691]:
# join zhvi data with states data because it's abbreviated in zhvi data
df_zhviCitiesJoined = pd.merge(df_zhviCities, df_states, left_on='State', right_on='Abbr State', how='left')

In [692]:
df_zhviCitiesJoined.head()

,RegionName,State_x,Zhvi,Abbr State,State_y
0,New York,NY,677000,NY,New York
1,Los Angeles,CA,686100,CA,California
2,Houston,TX,186000,TX,Texas
3,Chicago,IL,229400,IL,Illinois
4,San Antonio,TX,176100,TX,Texas


Now that we have joined the data, it's important to check for any NaN values to make sure our join worked correctly. Luckily, there are none, so we may proceed.

It's also important to remove un-needed columns and to update header names to make the data easier to work with.

In [693]:
# any joins that didn't work? nope
df_zhviCitiesJoined['State_y'].isnull().values.any()

False

In [694]:
# let's remove unneeded/duplicate columns
df_zhviCitiesJoined = df_zhviCitiesJoined.drop(columns = ['State_x', 'Abbr State'])

In [695]:
# update header names. RegionName is City--from the zillow site
df_zhviCitiesJoined = df_zhviCitiesJoined.rename(columns = {'RegionName': 'City', 'State_y': 'State'})

In [696]:
df_zhviCitiesJoined.head()

,City,Zhvi,State
0,New York,677000,New York
1,Los Angeles,686100,California
2,Houston,186000,Texas
3,Chicago,229400,Illinois
4,San Antonio,176100,Texas


In [697]:
# let's save over the old file
df_zhviCitiesJoined.to_csv('zhviCities.csv',header=1,index=False)

In [698]:
# let's load
df_zhviCities = pd.read_csv('zhviCities.csv')

### Okay, time to download (3) Wikipedia 'Household income in the United States' data.

In [701]:
html = wp.page('Household income in the United States').html().encode("UTF-8") # returns HTML

In [704]:
# pandas reads html tables into a list of DataFrame objects. perfect for what we're trying to do
pd.read_html(html)[:4]




[    0                                                  1
 0 NaN  This article needs to be updated. Please updat...,
                            Variable 1999 Previous Record 2007 Pre-Crisis Peak  \
 0  Real median household income[20]              $60,062              $59,534   
 
   2012 Post-Crisis Trough 2016 Previous Record 2017 Record             2018  
 0                 $54,569              $60,309     $61,372  Avail. Sept '19  ,
       Ethnic category Mean household income
 0         Asian alone              $114,105
 1         White alone               $89,632
 2  Hispanic or Latino               $68,319
 3               Black               $58,985,
     0                                                  1
 0 NaN  This article needs to be updated. Please updat...]

In [707]:
df = pd.read_html(html)[7] # set variable for the dataframe. the seventh element is our table of interest -- median household income

In [708]:
df.to_csv('medianIncome.csv',header=1,index=False) # save to csv

In [709]:
df_medianIncome = pd.read_csv('medianIncome.csv') # load csv

In [713]:
# let's only keep the columns we need, so take most recent household income value
df_medianIncome = df_medianIncome[['State', '2017']]

## Step II. Join the Data

### First, let's join (2) 'Zillow Home Value Index' data with (3) Wikipedia 'Household income in the United States' data to form a 'supplemental' data frame (the 'populous cities' data frame is the 'main' data frame).



In [716]:
# join zhvi data with income data--this is supplemental data, wikipedia populous cities data is the main data 

df_supplemental = pd.merge(df_zhviCities, df_medianIncome, on='State', how='left')

In [717]:
df_supplemental.head()

,City,Zhvi,State,2017
0,New York,677000,New York,"$64,894"
1,Los Angeles,686100,California,"$71,805"
2,Houston,186000,Texas,"$59,206"
3,Chicago,229400,Illinois,"$62,992"
4,San Antonio,176100,Texas,"$59,206"


In [718]:
df_supplemental = df_supplemental.rename(columns={'2017': 'Median Income'})

The join appears to have been properly executed, as there are no resulting NaN values.

In [719]:
# any joins that didn't work? nope

df_supplemental['Median Income'].isnull().values.any()

False

The median income field needs to be changed from an 'object' data type to a 'float' data type.

In [720]:
# are the data types correct? no, we need to update median income
df_supplemental.dtypes

City             object
Zhvi              int64
State            object
Median Income    object
dtype: object

In [721]:
df_supplemental['Median Income'] = df_supplemental['Median Income'].apply(lambda x: x.replace('$', '').replace(',', '')).astype('float')


In [722]:
df_supplemental.dtypes

City              object
Zhvi               int64
State             object
Median Income    float64
dtype: object

In [723]:
# save to csv
df_supplemental.to_csv('supplemental.csv',header=1,index=False) # save to csv

In [724]:
df_supplemental = pd.read_csv('supplemental.csv')

### Now, let's join the 'supplemental' data frame with the 'main' (1) Wikipedia 'List of United States cities by population' data frame.

In [726]:
df_topCitiesJoined = pd.merge(df_topCities, df_supplemental, on=['City', 'State'], how='left')

Unfortunately, there appears to be NaN values resulting from the join, so we will have to investigate what went wrong with our join.

In [729]:
# check for NaN values
df_topCitiesJoined[['Zhvi', 'Median Income']].isnull().values.any()

True

The following records did not join properly:

In [731]:
# which city/state combinations did not join well
df_topCitiesJoined[df_topCitiesJoined.isna().any(axis=1)]

,2018rank,City,State,2018estimate,2010Census,2016 land area (sq mi),2016 population density (sq mi),Zhvi,Median Income
0,1,New York City,New York,8398748,8175133,301.5,28317.0,NaN,NaN
19,20,"Washington, D.C.",District of Columbia,702455,601723,61.1,11148.0,NaN,NaN
22,23,Detroit,Michigan,672662,713777,138.8,4847.0,NaN,NaN
63,64,St. Louis,Missouri,302838,319294,62.0,5023.0,NaN,NaN
78,79,St. Petersburg,Florida,265098,244769,61.8,4223.0,NaN,NaN
88,89,WinstonâSalem,North Carolina,246328,229617,132.5,1828.0,NaN,NaN
106,107,Birmingham,Alabama,209880,212237,146.1,1452.0,NaN,NaN
124,123,Augusta,Georgia,196939,195844,302.5,652.0,NaN,NaN
125,124,Port St. Lucie,Florida,195248,164603,118.9,1557.0,NaN,NaN
134,133,Shreveport,Louisiana,188987,199311,107.1,1820.0,NaN,NaN


Below is code used to investigate the above records. It appears that the joins did not properly execute because: i) the 'City' value in the 'supplemental' table was spelled differently than the 'City' value in the 'main' table, or ii) the 'City' value in the 'main' table was not found in the 'supplemental' table. 

For case i, let's update the 'supplemental' table spelling to match the 'main' table spelling.

For case ii, let's take a 'supplemental' table city that's nearby the 'main' table city, and replace the 'supplemental' table city with the 'main' table city--we will essentially use a nearby city's median household income as an estimate for the 'main' table city's median household income.

In [732]:
# for new york, looks like we have to modify the supplemental 'New York' to 'New York City'
df_supplemental[df_supplemental['City'].str.find('New York') == 0]
# for washington, we'll have to modify 'Washington' to 'Washington, D.C.'
df_supplemental[df_supplemental['City'].str.find('Washington') == 0]
# 'detroit beach' to 'Detroit'
df_supplemental[df_supplemental['City'].str.find('Detroit') == 0]
# 'Saint Louis' to 'St. Louis'
df_supplemental[df_supplemental['City'].str.find('Saint') == 0]
# Saint Petersburg to 'St. Petersburg'
df_supplemental[df_supplemental['City'].str.find('Saint Petersburg') == 0]
# error in main table 'WinstonâSalem' to 'Winston-Salem'
df_supplemental[df_supplemental['City'].str.find('Winston') == 0]
# 'Homewood' to 'Birmingham' (Alabama)
df_supplemental[df_supplemental['State'].str.find('Alabama') == 0].sort_values('City') # no birmingham
df_supplemental[df_supplemental['City'].str.find('Homewood') == 0] # homewood is close
# 'Martinez' is close to 'Augusta' (Georgia)
df_supplemental[df_supplemental['City'].str.find('Martinez') == 0]
# 'Port Saint Lucie' to 'Port St. Lucie' (Florida)
df_supplemental[df_supplemental['City'].str.find('Port') == 0]
# 'Keithville' to 'Shreveport' (LA)
df_supplemental[df_supplemental['City'].str.find('Keithville') == 0]
# Richland to 'Jackson' (MS)
df_supplemental[df_supplemental['City'].str.find('Richland') == 0]
# 'Arlington' to Alexandria' (VA)
df_supplemental[df_supplemental['City'].str.find('Arlington') == 0]
# 'Glen Avon' to 'Jurupa Valley' (CA)
df_supplemental[df_supplemental['City'].str.find('Glen Avon') == 0]
# 'Georgetown' to 'South Bend' (IN)
df_supplemental[df_supplemental['City'].str.find('Georgetown') == 0]

,City,Zhvi,State,Median Income
478,Georgetown,297100,Texas,59206.0
1177,Georgetown,190800,Kentucky,48375.0
2221,Georgetown,135100,South Carolina,50570.0
4353,Georgetown,208700,Indiana,54181.0
4868,Georgetown,110200,Ohio,54021.0
5269,Georgetown,474100,Massachusetts,77385.0
7725,Georgetown,54200,Illinois,62992.0
8513,Georgetown,190000,Tennessee,51340.0
9817,Georgetown,364400,California,71805.0
11804,Georgetown,311300,Colorado,69117.0


In [733]:
# this code updates the data frames in response to the above join errors
df_supplemental.at[0, 'City'] = 'New York City'
df_supplemental.at[26, 'City'] = 'Washington, D.C.'
df_supplemental.at[10830, 'City'] = 'Detroit'
df_supplemental.at[67, 'City'] = 'St. Louis'
df_supplemental.at[94, 'City'] = 'St. Petersburg'
df_topCities.at[88, 'City'] = 'Winston-Salem' # df_topCities has error
df_supplemental.at[2145, 'City'] = 'Birmingham'
df_supplemental.at[1255, 'City'] = 'Augusta'
df_supplemental.at[146, 'City'] = 'Port St. Lucie'
df_supplemental.at[4373, 'City'] = 'Shreveport'
df_supplemental.at[6736, 'City'] = 'Jackson'
df_supplemental.at[117, 'City'] = 'Alexandria'
df_supplemental.at[3374, 'City'] = 'Jurupa Valley'
df_supplemental.at[4353, 'City'] = 'South Bend'

Let's rejoin the data based on our fixes. There are no join errors this time, so we are ready to move on to the next step!

In [734]:
# let's rejoin
df_topCitiesJoined = pd.merge(df_topCities, df_supplemental, on=['City', 'State'], how='left')

In [736]:
# any errors this time? no? great!
df_topCitiesJoined[['Zhvi', 'Median Income']].isnull().values.any()

False

## Step III. Analyze the Data

Now that we have joined the data, let's see which cities have the highest "Cost Score" (see top of notebook for "Cost Score" equation). 

In [738]:
# create 'Cost Score' field
df_topCitiesJoined['Cost Score'] = df_topCitiesJoined['Zhvi'] / df_topCitiesJoined['Median Income'] 

In [739]:
df_topCitiesJoined.head()

,2018rank,City,State,2018estimate,2010Census,2016 land area (sq mi),2016 population density (sq mi),Zhvi,Median Income,Cost Score
0,1,New York City,New York,8398748,8175133,301.5,28317.0,677000,64894.0,10.432397
1,2,Los Angeles,California,3990456,3792621,468.7,8484.0,686100,71805.0,9.555045
2,3,Chicago,Illinois,2705994,2695598,227.3,11900.0,229400,62992.0,3.641732
3,4,Houston,Texas,2325502,2100263,637.5,3613.0,186000,59206.0,3.141573
4,5,Phoenix,Arizona,1660272,1445632,517.6,3120.0,242600,56581.0,4.287658


Next, let's create a "Cost Rank", with the most costly "Cost Score" value ranked first.

In [740]:
# create Cost Rank column
df_topCitiesJoined['Cost Rank'] = df_topCitiesJoined['Cost Score'].rank(ascending=False)

In [746]:
df_topCitiesJoined.to_csv('topCitiesJoined.csv',header=1,index=False) # save to csv

Now, let's see which are the 10 costliest cities.

In [742]:
df_topCitiesJoined.nsmallest(10, 'Cost Rank')

,2018rank,City,State,2018estimate,2010Census,2016 land area (sq mi),2016 population density (sq mi),Zhvi,Median Income,Cost Score,Cost Rank
170,169,Sunnyvale,California,153185,140081,22.0,6944.0,1713500,71805.0,23.863241,1.0
295,294,San Mateo,California,105025,97207,12.1,8592.0,1385800,71805.0,19.299492,2.0
14,15,San Francisco,California,883305,805235,46.9,18569.0,1357500,71805.0,18.905369,3.0
215,214,Santa Clara,California,129488,116468,18.4,6845.0,1305100,71805.0,18.175615,4.0
233,232,Berkeley,California,121643,112580,10.5,11547.0,1217700,71805.0,16.958429,5.0
95,96,Fremont,California,237807,214089,77.5,3008.0,1120700,71805.0,15.607548,6.0
9,10,San Jose,California,1030119,945942,177.5,5777.0,1039700,71805.0,14.479493,7.0
301,299,Burbank,California,103695,103340,17.4,6003.0,992200,71805.0,13.817979,8.0
287,286,Daly City,California,107008,101123,7.6,14009.0,962500,71805.0,13.404359,9.0
178,177,Bellevue,Washington,147599,122363,33.5,4221.0,919800,70979.0,12.958762,10.0


Looks like 9 out of the 10 costliest cities are in California! And only one--San Jose--of the 10 most populous cities is on this list! If you're looking for affordable living, you may want to look outside of California, and check out the most affordable cities below!

In [748]:
df_topCitiesJoined.nlargest(10, 'Cost Rank')

,2018rank,City,State,2018estimate,2010Census,2016 land area (sq mi),2016 population density (sq mi),Zhvi,Median Income,Cost Score,Cost Rank
191,190,Dayton,Ohio,140640,141527,55.7,2522.0,51800,54021.0,0.958886,317.0
51,52,Cleveland,Ohio,383793,396815,77.7,4965.0,55300,54021.0,1.023676,316.0
110,111,Rochester,New York,206284,210565,35.8,5835.0,75500,64894.0,1.163436,315.0
121,120,Akron,Ohio,198006,199110,62.0,3188.0,67000,54021.0,1.240258,314.0
73,74,Toledo,Ohio,274975,287208,80.7,3451.0,69600,54021.0,1.288388,313.0
82,83,Buffalo,New York,256304,261310,40.4,6359.0,86200,64894.0,1.328320,312.0
186,185,Syracuse,New York,142749,145170,25.0,5735.0,87000,64894.0,1.340648,311.0
179,178,Rockford,Illinois,146526,152871,63.5,2325.0,86400,62992.0,1.371603,310.0
270,269,Peoria,Illinois,111388,115007,48.2,2371.0,87700,62992.0,1.392240,309.0
171,170,Macon,Georgia,153095,91351,249.3,612.0,78600,56183.0,1.399000,308.0
